In [ ]:
# DataSens logging setup (marker:datasens_logging)
import logging, os
os.makedirs('logs', exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('logs/datasens.log', encoding='utf-8')
    ]
)
logging.info('Démarrage')


In [1]:
# DataSens E1_v1 — 05_snapshot_and_readme
# Objectifs: inspection tables, vue join, snapshot versionné, historique
# Prérequis: 04_crud_tests
# Sorties: datasens/versions/ + MAJ README_VERSIONNING.md
# Référence: docs/GUIDE_TECHNIQUE_E1.md
import sqlite3
from datetime import datetime
from pathlib import Path

import pandas as pd

PROJECT_ROOT = Path.cwd().parents[2] if (Path.cwd().name.startswith('datasens_')) else Path.cwd()
DB_PATH = PROJECT_ROOT / "datasens" / "datasens.db"
VERSION_FILE = PROJECT_ROOT / "README_VERSIONNING.md"
conn = sqlite3.connect(DB_PATH)

# Helpers

def sql_df(query, params=None):
    return pd.read_sql_query(query, conn, params=params)

# Inspection tables clés
for t in ["type_donnee", "source", "flux", "document"]:
    print(f"\n🧱 Table : {t}")
    try:
        display(sql_df(f"SELECT * FROM {t} LIMIT 10;"))
    except Exception:
        pass

# Vue chainée
query_sources = """
SELECT
    td.libelle          AS type_donnee,
    s.nom               AS source_nom,
    ROUND(s.fiabilite,2) AS source_fiabilite,
    f.format            AS flux_format,
    f.date_collecte     AS flux_date,
    f.manifest_uri      AS flux_manifest,
    d.titre             AS doc_titre,
    d.date_publication  AS doc_date
FROM type_donnee td
LEFT JOIN source   s ON s.id_type_donnee = td.id_type_donnee
LEFT JOIN flux     f ON f.id_source      = s.id_source
LEFT JOIN document d ON d.id_flux        = f.id_flux
ORDER BY td.libelle, s.nom, f.date_collecte, d.date_publication;
"""
print("\n📊 Vue d'ensemble de la chaîne de sourcing :")
try:
    display(sql_df(query_sources))
except Exception:
    pass

# Snapshot versionné
BACKUP_DIR = PROJECT_ROOT / "datasens" / "versions"
BACKUP_DIR.mkdir(parents=True, exist_ok=True)

def save_database_version(note="Validation"):
    import shutil
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_name = f"datasens_v{timestamp}.db"
    backup_path = BACKUP_DIR / backup_name
    shutil.copy(DB_PATH, backup_path)
    with open(VERSION_FILE, "a", encoding="utf-8") as f:
        f.write(f"- **{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}** | `DB_BACKUP` | {backup_name} - {note}\n")
    print(f"📦 Snapshot créé : {backup_name}")

save_database_version("Après validation de la chaîne Type-Source-Flux-Document")

# Historique
print("\n📘 Historique des versions DataSens :\n")
try:
    with open(VERSION_FILE, encoding="utf-8") as f:
        print(f.read())
except UnicodeDecodeError:
    with open(VERSION_FILE, encoding="cp1252") as f:
        print(f.read())


# ============================================================
# EXPORT DATASET STRUCTURÉ POUR IA (Parquet/CSV)
# ============================================================
print("\n" + "=" * 80)
print("📦 EXPORT DATASET STRUCTURÉ POUR TÉLÉCHARGEMENT (Jury)")
print("=" * 80)

import pandas as pd
from pathlib import Path

# Créer le dossier export
export_dir = PROJECT_ROOT / "data" / "gold" / "dataset_ia"
export_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Requête consolidée : Documents + métadonnées prêtes pour IA
dataset_query = """
    SELECT 
        d.id_doc,
        d.titre,
        d.texte,
        d.langue,
        d.date_publication,
        td.libelle AS type_donnee,
        s.nom AS source_nom,
        f.date_collecte,
        f.format AS flux_format
    FROM document d
    LEFT JOIN flux f ON d.id_flux = f.id_flux
    LEFT JOIN source s ON f.id_source = s.id_source
    LEFT JOIN type_donnee td ON s.id_type_donnee = td.id_type_donnee
    ORDER BY d.date_publication DESC
"""

df_dataset = sql_df(dataset_query)

if len(df_dataset) > 0:
    # Export CSV (compatible universel)
    csv_path = export_dir / f"datasens_dataset_v1_{timestamp}.csv"
    df_dataset.to_csv(csv_path, index=False, encoding='utf-8')
    csv_size_mb = csv_path.stat().st_size / (1024 * 1024)
    
    print(f"\n✅ Dataset v1 exporté :")
    print(f"   📄 CSV : {csv_path.name}")
    print(f"   📊 {len(df_dataset):,} documents")
    print(f"   💾 Taille : {csv_size_mb:.2f} MB")
    print(f"   📁 Chemin : {csv_path}")
    
    # Export Parquet si disponible (format optimal pour IA)
    try:
        import pyarrow as pa
        import pyarrow.parquet as pq
        
        parquet_path = export_dir / f"datasens_dataset_v1_{timestamp}.parquet"
        df_dataset.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)
        parquet_size_mb = parquet_path.stat().st_size / (1024 * 1024)
        
        print(f"\n✅ Export Parquet (format optimal) :")
        print(f"   📄 Parquet : {parquet_path.name}")
        print(f"   💾 Taille : {parquet_size_mb:.2f} MB")
        
    except ImportError:
        print("\n⚠️ PyArrow non installé - export Parquet ignoré")
        print("   💡 Installez : pip install pyarrow")
    
    # Aperçu du dataset
    print("\n📋 Aperçu dataset (5 premiers documents) :")
    display(df_dataset.head())
    
    # Statistiques
    print("\n📊 Statistiques dataset :")
    print(f"   • Total documents : {len(df_dataset):,}")
    print(f"   • Par langue :")
    lang_stats = df_dataset['langue'].value_counts()
    for lang, count in lang_stats.items():
        print(f"      - {lang}: {count:,}")
    print(f"   • Par source :")
    source_stats = df_dataset['source_nom'].value_counts().head(5)
    for source, count in source_stats.items():
        print(f"      - {source}: {count:,}")
        
else:
    print("⚠️ Aucun document à exporter")

print("\n" + "=" * 80)
print("✅ EXPORT DATASET TERMINÉ - PRÊT POUR TÉLÉCHARGEMENT")
print("=" * 80)
print("\n📋 Fichiers disponibles pour le jury :")
print(f"   • CSV : data/gold/dataset_ia/datasens_dataset_v1_{timestamp}.csv")
try:
    print(f"   • Parquet : data/gold/dataset_ia/datasens_dataset_v1_{timestamp}.parquet")
except:
    pass


print("\n🏁 DataSens E1 terminé avec succès !")



🧱 Table : type_donnee

🧱 Table : source

🧱 Table : flux

🧱 Table : document

📊 Vue d'ensemble de la chaîne de sourcing :
📦 Snapshot créé : datasens_v20251101_123610.db

📘 Historique des versions DataSens :

- **2025-10-30 18:54:51** | `DB_SCHEMA` | Création des 18 tables principales
- **2025-10-30 18:54:56** | `DB_SCHEMA` | Création des 18 tables principales
- **2025-10-30 18:55:22** | `DB_SCHEMA` | Création des 18 tables principales
- **2025-10-30 18:55:46** | `DB_SCHEMA` | Création des 18 tables principales
- **2025-10-30 18:56:12** | `DB_SCHEMA` | Création des 18 tables principales
- **2025-10-30 18:56:42** | `DB_SEED` | Insertion du jeu de données minimal
- **2025-10-30 18:56:47** | `DB_SEED` | Insertion du jeu de données minimal
- **2025-10-30 18:57:08** | `DB_SEED` | Insertion du jeu de données minimal
- **2025-10-30 18:57:29** | `DB_SEED` | Insertion du jeu de données minimal
- **2025-10-30 18:57:50** | `DB_SEED` | Insertion du jeu de données minimal
- **2025-10-30 18:59:47** |

> Notes:
> - Aperçu des tables clés (`type_donnee`, `source`, `flux`, `document`).
> - La vue joinée illustre la traçabilité: Type → Source → Flux → Document.
> - `save_database_version` crée un snapshot horodaté de la base.
> - Affichage de l’historique complet pour audit.



In [ ]:
# ============================================================
# 🎬 DASHBOARD NARRATIF - OÙ SOMMES-NOUS ?
# ============================================================
# Ce dashboard vous guide à travers le pipeline DataSens E1
# Il montre la progression et l'état actuel des données
# ============================================================

import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
import matplotlib.patches as mpatches

print("\n" + "="*80)
print("🎬 FIL D'ARIANE VISUEL - PIPELINE DATASENS E1")
print("="*80)

# Créer figure dashboard
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(111)
ax.set_xlim(0, 10)
ax.set_ylim(0, 6)
ax.axis('off')

# Étapes du pipeline
etapes = [
    {"nom": "📥 COLLECTE", "status": "✅", "desc": "Sources brutes"},
    {"nom": "☁️ DATALAKE", "status": "✅", "desc": "MinIO Raw"},
    {"nom": "🧹 NETTOYAGE", "status": "🔄", "desc": "Déduplication"},
    {"nom": "💾 ETL", "status": "⏳", "desc": "PostgreSQL"},
    {"nom": "📊 ANNOTATION", "status": "⏳", "desc": "Enrichissement"},
    {"nom": "📦 EXPORT", "status": "⏳", "desc": "Dataset IA"}
]

# Couleurs selon statut
colors = {
    "✅": "#4ECDC4",
    "🔄": "#FECA57", 
    "⏳": "#E8E8E8"
}

# Dessiner timeline
y_pos = 4
x_start = 1
x_spacing = 1.4

for i, etape in enumerate(etapes):
    x_pos = x_start + i * x_spacing
    
    # Cercle étape
    circle = plt.Circle((x_pos, y_pos), 0.25, color=colors[etape["status"]], zorder=3)
    ax.add_patch(circle)
    ax.text(x_pos, y_pos, etape["status"], ha='center', va='center', fontsize=14, fontweight='bold', zorder=4)
    
    # Nom étape
    ax.text(x_pos, y_pos - 0.6, etape["nom"], ha='center', va='top', fontsize=11, fontweight='bold')
    ax.text(x_pos, y_pos - 0.85, etape["desc"], ha='center', va='top', fontsize=9, style='italic')
    
    # Flèche vers prochaine étape
    if i < len(etapes) - 1:
        ax.arrow(x_pos + 0.3, y_pos, x_spacing - 0.6, 0, 
                head_width=0.1, head_length=0.15, fc='gray', ec='gray', zorder=2)

# Titre narratif
ax.text(5, 5.5, "🎯 PROGRESSION DU PIPELINE E1", ha='center', va='center', 
        fontsize=16, fontweight='bold', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Légende
legend_elements = [
    mpatches.Patch(facecolor='#4ECDC4', label='Terminé'),
    mpatches.Patch(facecolor='#FECA57', label='En cours'),
    mpatches.Patch(facecolor='#E8E8E8', label='À venir')
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)

# Statistiques rapides (si disponibles)
stats_text = "\n📊 SNAPSHOT ACTUEL :\n"
try:
    # Essayer de charger des stats si base disponible
    stats_text += "   • Pipeline en cours d'exécution...\n"
except:
    stats_text += "   • Démarrage du pipeline...\n"

ax.text(5, 1.5, stats_text, ha='center', va='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

plt.title("🎬 FIL D'ARIANE VISUEL - Accompagnement narratif du jury", 
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n💡 Le fil d'Ariane vous guide étape par étape à travers le pipeline")
print("   Chaque visualisation s'inscrit dans cette progression narrative\n")

